In [ ]:
import nltk
import json
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
import warnings
warnings.filterwarnings('ignore')

In [ ]:
intents ={
  "intents": [
    {
      "tag": "aloe_vera",
      "patterns": [
        "What is Aloe Vera?",
        "What are the benefits of Aloe Vera?",
        "How do I use Aloe Vera?",
        "Are there any side effects of Aloe Vera?",
        "Can Aloe Vera help with skin issues?"
      ],
      "responses": [
        "Aloe Vera is a succulent plant known for its medicinal and cosmetic uses. It has soothing, moisturizing, and healing properties.",
        "The benefits of Aloe Vera include improved skin hydration, reduced inflammation, and support for digestion.",
        "Aloe Vera can be used topically for skin care or consumed in the form of supplements or juices.",
        "Some people may experience skin irritation or allergic reactions. It's best to do a patch test before using it extensively.",
        "Yes, Aloe Vera can help with skin issues such as burns, acne, and dryness due to its healing and moisturizing properties."
      ]
    },
    {
      "tag": "tulsi",
      "patterns": [
        "What is Tulsi?",
        "What are the health benefits of Tulsi?",
        "How can I use Tulsi?",
        "Are there any side effects of Tulsi?",
        "Can Tulsi help with stress?"
      ],
      "responses": [
        "Tulsi, also known as Holy Basil, is an aromatic plant used in traditional medicine for its therapeutic properties.",
        "Tulsi is known for its benefits in reducing stress, boosting immunity, and improving respiratory health.",
        "Tulsi can be consumed as tea, used in cooking, or applied topically in some forms.",
        "Tulsi is generally safe but might interact with certain medications. Consult a healthcare provider if you have any concerns.",
        "Yes, Tulsi is known to have adaptogenic properties that help in reducing stress and balancing the body's response to stress."
      ]
    },
    {
      "tag": "ashwagandha",
      "patterns": [
        "What is Ashwagandha?",
        "What are the benefits of Ashwagandha?",
        "How should I take Ashwagandha?",
        "Are there any side effects of Ashwagandha?",
        "Can Ashwagandha help with sleep?"
      ],
      "responses": [
        "Ashwagandha is an adaptogenic herb used in Ayurvedic medicine to help the body cope with stress and improve overall health.",
        "Benefits of Ashwagandha include stress reduction, improved energy levels, and enhanced cognitive function.",
        "Ashwagandha can be taken as a supplement in various forms, including capsules, powders, or teas.",
        "Some people may experience mild side effects such as digestive upset. It's best to start with a lower dose and consult a healthcare provider.",
        "Yes, Ashwagandha is often used to support better sleep and improve sleep quality due to its calming effects."
      ]
    },
    {
      "tag": "neem",
      "patterns": [
        "What is Neem?",
        "What are the uses of Neem?",
        "How can I use Neem?",
        "Are there any side effects of Neem?",
        "Can Neem help with acne?"
      ],
      "responses": [
        "Neem is a tree known for its medicinal properties, often used in traditional medicine for its antibacterial and anti-inflammatory effects.",
        "Neem can be used for various purposes, including skin care, dental health, and as a natural pesticide.",
        "Neem can be applied topically in the form of creams or oils, or taken internally as supplements.",
        "Neem is generally safe for most people, but it can cause some gastrointestinal issues in high doses. Always consult with a healthcare provider if unsure.",
        "Yes, Neem is known for its antibacterial properties, making it effective in treating acne and other skin infections."
      ]
    },
    {
      "tag": "aloe_vera_uses",
      "patterns": [
        "What are the uses of Aloe Vera?",
        "How can I use Aloe Vera for skin?",
        "Can Aloe Vera be used for digestive issues?",
        "What are some common uses of Aloe Vera?",
        "How is Aloe Vera used in cosmetics?"
      ],
      "responses": [
        "Aloe Vera can be used for soothing burns, moisturizing the skin, treating acne, and improving digestion. It's also used in various cosmetic products for its hydrating properties.",
        "For skin care, Aloe Vera gel can be applied topically to treat burns, cuts, and dry skin.",
        "Aloe Vera juice can aid digestion and support gut health when consumed in moderation.",
        "Common uses include treating minor skin irritations, adding moisture to the skin, and promoting healing of wounds.",
        "In cosmetics, Aloe Vera is used in lotions, creams, and masks for its moisturizing and soothing effects."
      ]
    },
    {
      "tag": "aloe_vera_advantages_disadvantages",
      "patterns": [
        "What are the advantages of Aloe Vera?",
        "What are the disadvantages of Aloe Vera?",
        "Are there any risks of using Aloe Vera?",
        "How does Aloe Vera benefit health?",
        "Can Aloe Vera have side effects?"
      ],
      "responses": [
        "Advantages of Aloe Vera include its ability to soothe skin irritations, improve skin hydration, and support digestive health. It's also known for its anti-inflammatory and antimicrobial properties.",
        "Disadvantages may include potential skin irritation or allergic reactions in some individuals. Overuse of Aloe Vera internally may also lead to digestive issues.",
        "Risks include potential allergic reactions and interactions with certain medications. It's best to use Aloe Vera with caution and consult a healthcare provider if unsure.",
        "Health benefits include reduced inflammation, improved skin healing, and enhanced digestion.",
        "Side effects can include mild irritation or allergic reactions, especially with topical use. Always perform a patch test before extensive use."
      ]
    },
    {
      "tag": "tulsi_uses",
      "patterns": [
        "What are the uses of Tulsi?",
        "How can Tulsi benefit health?",
        "Can Tulsi be used for stress relief?",
        "What are some common uses of Tulsi?",
        "How is Tulsi used in traditional medicine?"
      ],
      "responses": [
        "Tulsi, or Holy Basil, is used for stress relief, boosting immunity, and improving respiratory health. It's commonly consumed as tea, used in cooking, or applied topically.",
        "For stress relief, Tulsi tea can be consumed daily. Tulsi leaves can also be used in cooking for their health benefits.",
        "Common uses include making Tulsi tea for immune support, adding Tulsi leaves to dishes, or using Tulsi oil for topical applications.",
        "In traditional medicine, Tulsi is used to balance the body's systems, support respiratory function, and reduce stress."
      ]
    },
    {
      "tag": "tulsi_advantages_disadvantages",
      "patterns": [
        "What are the advantages of Tulsi?",
        "What are the disadvantages of Tulsi?",
        "Are there any risks of using Tulsi?",
        "How does Tulsi benefit health?",
        "Can Tulsi have side effects?"
      ],
      "responses": [
        "Advantages of Tulsi include its ability to reduce stress, boost immunity, and support respiratory health. It also has anti-inflammatory and antioxidant properties.",
        "Disadvantages may include potential interactions with medications and mild gastrointestinal issues. It's best to consult with a healthcare provider before extensive use.",
        "Risks include potential interactions with blood-thinning medications and digestive upset in some individuals.",
        "Health benefits include improved stress management, enhanced immunity, and support for respiratory health.",
        "Side effects are generally rare but may include mild digestive discomfort or interactions with certain medications."
      ]
    },
    {
      "tag": "ashwagandha_uses",
      "patterns": [
        "What are the uses of Ashwagandha?",
        "How can Ashwagandha benefit health?",
        "Can Ashwagandha be used for anxiety?",
        "What are some common uses of Ashwagandha?",
        "How is Ashwagandha used in supplements?"
      ],
      "responses": [
        "Ashwagandha is used for reducing stress, improving energy levels, supporting cognitive function, and enhancing overall vitality. It’s commonly taken as a supplement in capsules, powders, or teas.",
        "For anxiety, Ashwagandha can be taken as a supplement to help manage stress and promote relaxation.",
        "Common uses include taking Ashwagandha supplements for stress relief, improved sleep, and enhanced cognitive function.",
        "In supplements, Ashwagandha is available in various forms such as capsules, powders, and extracts."
      ]
    },
    {
      "tag": "ashwagandha_advantages_disadvantages",
      "patterns": [
        "What are the advantages of Ashwagandha?",
        "What are the disadvantages of Ashwagandha?",
        "Are there any risks of using Ashwagandha?",
        "How does Ashwagandha benefit health?",
        "Can Ashwagandha have side effects?"
      ],
      "responses": [
        "Advantages of Ashwagandha include its ability to reduce stress, enhance cognitive function, and improve overall vitality. It's known for its adaptogenic properties and support for better sleep.",
        "Disadvantages may include potential gastrointestinal issues, interactions with medications, and mild drowsiness in some individuals.",
        "Risks include potential interactions with sedatives and thyroid medications. It's best to use Ashwagandha under the guidance of a healthcare provider.",
        "Health benefits include reduced stress, improved sleep quality, and enhanced cognitive function.",
        "Side effects can include digestive upset, drowsiness, or interactions with certain medications. Consult a healthcare provider for personalized advice."
      ]
    },
    {
      "tag": "neem_uses",
      "patterns": [
        "What are the uses of Neem?",
        "How can Neem benefit health?",
        "Can Neem be used for skin care?",
        "What are some common uses of Neem?",
        "How is Neem used in traditional medicine?"
      ],
      "responses": [
        "Neem is used for its antibacterial, antifungal, and anti-inflammatory properties. Common uses include treating skin conditions, improving oral health, and acting as a natural pesticide.",
        "For skin care, Neem can be used topically as an oil or in creams to treat acne and other skin issues.",
        "Common uses include using Neem oil for skin and hair care, Neem sticks for dental health, and Neem extracts as natural pesticides.",
        "In traditional medicine, Neem is used to treat a variety of conditions including skin infections, digestive issues, and as a general health tonic."
      ]
    },
    {
      "tag": "neem_advantages_disadvantages",
      "patterns": [
        "What are the advantages of Neem?",
        "What are the disadvantages of Neem?",
        "Are there any risks of using Neem?",
        "How does Neem benefit health?",
        "Can Neem have side effects?"
      ],
      "responses": [
        "Advantages of Neem include its antibacterial, antifungal, and anti-inflammatory properties, making it effective for skin care, dental health, and as a natural pesticide.",
        "Disadvantages may include potential skin irritation or allergic reactions in some individuals. Neem can also have a strong odor which some people find unpleasant.",
        "Risks include potential allergic reactions or gastrointestinal issues if consumed in large amounts. It's best to consult with a healthcare provider if using Neem extensively.",
        "Health benefits include improved skin health, enhanced oral hygiene, and natural pest control.",
        "Side effects can include mild skin irritation, digestive upset, or allergic reactions in some people. Use Neem products cautiously and consult a healthcare provider if needed."
      ]
    }
  ]
}


In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
stemmer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?']


for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # Add to our words list
        words.extend(w)
        # Add to documents in our corpus
        documents.append((w, intent['tag']))
        # Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


words = [stemmer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))


classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

60 documents
12 classes ['aloe_vera', 'aloe_vera_advantages_disadvantages', 'aloe_vera_uses', 'ashwagandha', 'ashwagandha_advantages_disadvantages', 'ashwagandha_uses', 'neem', 'neem_advantages_disadvantages', 'neem_uses', 'tulsi', 'tulsi_advantages_disadvantages', 'tulsi_uses']
51 unique lemmatized words ['acne', 'advantage', 'aloe', 'anxiety', 'any', 'are', 'ashwagandha', 'be', 'benefit', 'can', 'care', 'common', 'cosmetic', 'digestive', 'disadvantage', 'do', 'doe', 'effect', 'for', 'have', 'health', 'help', 'how', 'i', 'in', 'is', 'issue', 'medicine', 'neem', 'of', 'relief', 'risk', 'should', 'side', 'skin', 'sleep', 'some', 'stress', 'supplement', 'take', 'the', 'there', 'traditional', 'tulsi', 'us', 'use', 'used', 'using', 'vera', 'what', 'with']


In [ ]:
training = []
output = []
output_empty = [0] * len(classes)

# Training set, bag of words for each sentence
for doc in documents:
    # Initialize our bag of words
    bag = []
    # List of tokenized words for the pattern
    pattern_words = doc[0]
    # Lemmatize each word
    pattern_words = [stemmer.lemmatize(word.lower()) for word in pattern_words]
    # Create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Output is a '0' for each tag and '1' for the current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)


def synonym_replacement(tokens, limit):
    augmented_sentences = []
    for i in range(len(tokens)):
        synonyms = []
        for syn in wordnet.synsets(tokens[i]):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
        if len(synonyms) > 0:
            num_augmentations = min(limit, len(synonyms))
            sampled_synonyms = random.sample(synonyms, num_augmentations)
            for synonym in sampled_synonyms:
                augmented_tokens = tokens[:i] + [synonym] + tokens[i + 1:]
                augmented_sentences.append(' '.join(augmented_tokens))
    return augmented_sentences


# Augment the training data using synonym replacement
augmented_data = []
limit_per_tag = 100

for i, doc in enumerate(training):
    bag, output_row = doc
    tokens = [words[j] for j in range(len(words)) if bag[j] == 1]
    augmented_sentences = synonym_replacement(tokens, limit_per_tag)
    for augmented_sentence in augmented_sentences:
        augmented_bag = [1 if augmented_sentence.find(word) >= 0 else 0 for word in words]
        augmented_data.append([augmented_bag, output_row])

# Combine the original and augmented data
combined_data = training + augmented_data

random.shuffle(combined_data)


In [ ]:
from sklearn.model_selection import train_test_split


def separate_data_by_tags(data):
    data_by_tags = {}
    for d in data:
        tag = tuple(d[1])
        if tag not in data_by_tags:
            data_by_tags[tag] = []
        data_by_tags[tag].append(d)
    return data_by_tags.values()


separated_data = separate_data_by_tags(combined_data)

# Lists to store training and testing data
training_data = []
testing_data = []

# Split each tag's data into training and testing sets
for tag_data in separated_data:
    train_data, test_data = train_test_split(tag_data, test_size=0.2, random_state=42)
    training_data.extend(train_data)
    testing_data.extend(test_data)


random.shuffle(training_data)
random.shuffle(testing_data)

# Convert training and testing data back to np.array
train_x = np.array([d[0] for d in training_data])
train_y = np.array([d[1] for d in training_data])
test_x = np.array([d[0] for d in testing_data])
test_y = np.array([d[1] for d in testing_data])

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(0.2)

        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.2)

        self.fc3 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.bn1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.relu2(x)
        x = self.bn2(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        output = self.softmax(x)
        return output

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return self.softmax(x)

class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

def accuracy(predictions, targets):
    predicted_labels = torch.argmax(predictions, dim=1)
    true_labels = torch.argmax(targets, dim=1)
    correct = (predicted_labels == true_labels).sum().item()
    total = targets.size(0)
    return correct / total

def test_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = len(test_loader)

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item() * inputs.size(0)
            total_accuracy += accuracy(outputs, targets) * inputs.size(0)

    average_loss = total_loss / len(test_loader.dataset)
    average_accuracy = total_accuracy / len(test_loader.dataset)
    return average_loss, average_accuracy

In [ ]:
# Create DataLoader for training and testing data
train_x = torch.tensor(train_x).float()
train_y = torch.tensor(train_y).float()
test_x = torch.tensor(test_x).float()
test_y = torch.tensor(test_y).float()

batch_size = 64
train_dataset = CustomDataset(train_x, train_y)
test_dataset = CustomDataset(test_x, test_y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the model, loss function, and optimizer
input_size = len(train_x[0])
hidden_size = 8
output_size = len(train_y[0])
model = NeuralNetwork(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Train the model and evaluate on the testing set
num_epochs = 50
for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    for inputs, targets in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update statistics
        running_loss += loss.item() * inputs.size(0)
        running_acc += accuracy(outputs, targets) * inputs.size(0)

    # Calculate average training loss and accuracy for the epoch
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_acc / len(train_loader.dataset)

    # Print training loss and accuracy for each epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_acc:.4f}")

    # Evaluate on the testing set
    test_loss, test_accuracy = test_model(model, test_loader, criterion)
    print(f"Epoch [{epoch+1}/{num_epochs}], Testing Loss: {test_loss:.4f}, Testing Accuracy: {test_accuracy:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'model.pth')

Epoch [1/50], Training Loss: 0.2863, Training Accuracy: 0.0848
Epoch [1/50], Testing Loss: 0.2828, Testing Accuracy: 0.1296
Epoch [2/50], Training Loss: 0.2763, Training Accuracy: 0.2077
Epoch [2/50], Testing Loss: 0.2649, Testing Accuracy: 0.2539
Epoch [3/50], Training Loss: 0.2481, Training Accuracy: 0.2940
Epoch [3/50], Testing Loss: 0.2294, Testing Accuracy: 0.3250
Epoch [4/50], Training Loss: 0.2139, Training Accuracy: 0.3922
Epoch [4/50], Testing Loss: 0.1980, Testing Accuracy: 0.5329
Epoch [5/50], Training Loss: 0.1823, Training Accuracy: 0.6185
Epoch [5/50], Testing Loss: 0.1660, Testing Accuracy: 0.6594
Epoch [6/50], Training Loss: 0.1496, Training Accuracy: 0.6997
Epoch [6/50], Testing Loss: 0.1340, Testing Accuracy: 0.7544
Epoch [7/50], Training Loss: 0.1188, Training Accuracy: 0.7952
Epoch [7/50], Testing Loss: 0.1060, Testing Accuracy: 0.8307
Epoch [8/50], Training Loss: 0.0932, Training Accuracy: 0.8695
Epoch [8/50], Testing Loss: 0.0842, Testing Accuracy: 0.8777
Epoch [9

In [ ]:
def load_model(model_path, input_size, hidden_size, output_size):
    model = NeuralNetwork(input_size, hidden_size, output_size)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# Function to preprocess the input sentence
def preprocess_sentence(sentence, words):
    sentence_words = sentence.lower().split()
    sentence_words = [word for word in sentence_words if word in words]
    return sentence_words

# Function to convert the preprocessed sentence into a feature vector
def sentence_to_features(sentence_words, words):
    features = [1 if word in sentence_words else 0 for word in words]
    return torch.tensor(features).float().unsqueeze(0)

# Function to generate a response using the trained model
def generate_response(sentence, model, words, classes):
    sentence_words = preprocess_sentence(sentence, words)
    if len(sentence_words) == 0:
        return "I'm sorry, but I don't understand. Can you please rephrase or provide more information?"

    features = sentence_to_features(sentence_words, words)
    with torch.no_grad():
        outputs = model(features)

    probabilities, predicted_class = torch.max(outputs, dim=1)
    confidence = probabilities.item()
    predicted_tag = classes[predicted_class.item()]

    if confidence > 0.5:
        for intent in intents['intents']:
            if intent['tag'] == predicted_tag:
                return random.choice(intent['responses'])

    return "I'm sorry, but I'm not sure how to respond to that."

In [ ]:
model_path = 'model.pth'
input_size = len(words)
hidden_size = 8
output_size = len(classes)
model = load_model(model_path, input_size, hidden_size, output_size)

# Test the chatbot response
print('Hello! I am a chatbot. How can I help you today? Type "quit" to exit.')
while True:
    user_input = input('> ')
    if user_input.lower() == 'quit':
        break
    response = generate_response(user_input, model, words, classes)
    print(response)

Hello! I am a chatbot. How can I help you today? Type "quit" to exit.
> hi tell me about aloe vera
Some people may experience skin irritation or allergic reactions. It's best to do a patch test before using it extensively.
> what is the use of aloe vera
Yes, Aloe Vera can help with skin issues such as burns, acne, and dryness due to its healing and moisturizing properties.
> tell me about tulsi
Tulsi is known for its benefits in reducing stress, boosting immunity, and improving respiratory health.
> disadvantage of tulsi
Health benefits include improved stress management, enhanced immunity, and support for respiratory health.
> use of tulsi
Side effects are generally rare but may include mild digestive discomfort or interactions with certain medications.
> quit
